In [5]:
!pip install -q transformers datasets accelerate

In [7]:
!pip install --upgrade accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 7.2 MB/s eta 0:00:00:00:01
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.2.1
    Uninstalling accelerate-1.2.1:
      Successfully uninstalled accelerate-1.2.1


In [3]:
!pip install faiss-gpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 19.9 MB/s eta 0:00:00:00:0100:01


In [31]:
import torch
import numpy as np
import pandas as pd
import os
import faiss
from datasets import Dataset
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, TrainingArguments, Trainer
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import AutoModelForCausalLM

In [5]:
data = pd.read_csv('/kaggle/input/medicaldata/finaldata.csv')
data['question'] = data['question'].fillna("").astype(str)
data['answer'] = data['answer'].fillna("").astype(str)

questions = data['question'].tolist() 
answers = data['answer'].tolist()     

In [ ]:
# torch.cuda.empty_cache()
# torch.cuda.ipc_collect()

# RAG with Fallback on LLM

In [56]:
pip install sacremoses


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 14.6 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [57]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load BioGPT
fallback_tokenizer = AutoTokenizer.from_pretrained("microsoft/biogpt")
fallback_model = AutoModelForCausalLM.from_pretrained("microsoft/biogpt")

pytorch_model.bin:   0%|          | 0.00/1.56G [00:00<?, ?B/s]

In [ ]:

retrieval_model = SentenceTransformer('all-MiniLM-L6-v2')


**Embedding with Faiss**

In [33]:
questions = data['question'].tolist() 
answers = data['answer'].tolist()
question_embeddings = retrieval_model.encode(questions, convert_to_numpy=True)
dimension = question_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(question_embeddings)

Batches:   0%|          | 0/319 [00:00<?, ?it/s]

In [58]:
def generate_fallback_answer(query):
    inputs = fallback_tokenizer(
        query, return_tensors="pt", max_length=128, truncation=True, padding="max_length"
    )
    outputs = fallback_model.generate(inputs["input_ids"], max_new_tokens=50, num_beams=5, early_stopping=True)
    return fallback_tokenizer.decode(outputs[0], skip_special_tokens=True)

    
threshold = 0.8  # Distance threshold
def query_pipeline(user_query):
    query_embedding = retrieval_model.encode([user_query], convert_to_numpy=True)
    k = 1
    distances, indices = index.search(query_embedding, k)
    distance = distances[0][0]
    retrieved_answer = answers[indices[0][0]]

    # Fallback logic
    if distance > threshold:
        return generate_fallback_answer(user_query)  # Use BioMedLM for fallback
    else:
        return retrieved_answer

# Inference

In [59]:
def interactive_test():
    print("Interactive Doctor GPT Chatbot")
    print("Type 'exit' to quit.\n")
    while True:
        user_query = input("Enter your question: ")
        if user_query.lower() == 'exit':
            print("Exiting interactive session. Goodbye!")
            break
        response = query_pipeline(user_query)
        print("\n**Response:**", response, "\n")

# Run the interactive session
interactive_test()


Interactive Doctor GPT Chatbot
Type 'exit' to quit.



Enter your question:  I am having fever with swelling on hands.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


**Response:** Hydroxyurea 



Enter your question:  What are the symptoms of covid infection?


...

Enter your question:  A 35-year-old woman with a history of Crohn disease presents for a follow-up appointment. She says that lately, she has started to notice difficulty walking. She says that some of her friends have joked that she appears to be walking as if she was drunk. Past medical history is significant for Crohn disease diagnosed 2 years ago, managed with natalizumab for the past year because her intestinal symptoms have become severe and unresponsive to other therapies. On physical examination, there is gait and limb ataxia present. Strength is 4/5 in the right upper limb. A T1/T2 MRI of the brain is ordered and is shown. Which of the following is the most likely diagnosis?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


**Response:** Progressive multifocal encephalopathy (PML) 



Enter your question:  exit


Exiting interactive session. Goodbye!


# Streamlit app code

In [67]:
!pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 27.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 84.3 MB/s eta 0:00:00:00:0100:01


In [68]:
import subprocess

In [69]:
streamlit_code = """
import streamlit as st
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM
import faiss
import pandas as pd

# Load RAG components
retrieval_model = SentenceTransformer('all-MiniLM-L6-v2')  # Sentence embeddings
index = faiss.read_index("/kaggle/working/faiss_index.index")  # FAISS index
data = pd.read_csv('/kaggle/input/medicaldata/finaldata.csv')
questions = data['question'].tolist()
answers = data['answer'].tolist()

# Load BioGPT for fallback
fallback_tokenizer = AutoTokenizer.from_pretrained("microsoft/biogpt")
fallback_model = AutoModelForCausalLM.from_pretrained("microsoft/biogpt")

# Add padding token if missing
if fallback_tokenizer.pad_token_id is None:
    fallback_tokenizer.pad_token = '[PAD]'
    fallback_tokenizer.pad_token_id = fallback_tokenizer.convert_tokens_to_ids('[PAD]')
    fallback_model.resize_token_embeddings(len(fallback_tokenizer))

# Function to retrieve answers using RAG
def retrieve_answer(query):
    query_embedding = retrieval_model.encode([query], convert_to_numpy=True)
    k = 1  # Number of nearest neighbors
    distances, indices = index.search(query_embedding, k)
    distance = distances[0][0]
    retrieved_answer = answers[indices[0][0]]
    return retrieved_answer, distance

# Function to generate fallback answer using BioGPT
def generate_fallback_answer(query):
    inputs = fallback_tokenizer(
        query, return_tensors="pt", max_length=128, truncation=True, padding="max_length"
    )
    outputs = fallback_model.generate(
        inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=50,
        num_beams=5,
        early_stopping=True,
        pad_token_id=fallback_tokenizer.pad_token_id
    )
    return fallback_tokenizer.decode(outputs[0], skip_special_tokens=True)

# Query pipeline combining RAG and fallback
threshold = 0.8  # Distance threshold for fallback
def query_pipeline(user_query):
    retrieved_answer, distance = retrieve_answer(user_query)
    if distance > threshold:
        return generate_fallback_answer(user_query)  # Use fallback if RAG fails
    else:
        return retrieved_answer  # Use RAG answer if within threshold

# Streamlit app
def main():
    st.title("Doctor GPT")
    st.markdown(
        "This bot uses RAG for known queries and BioGPT for fallback logic."
    )

    # Initialize chat history
    if "messages" not in st.session_state:
        st.session_state.messages = []  # To store the chat history

    # Sidebar for clearing chat
    with st.sidebar:
        if st.button("Clear Chat"):
            st.session_state.messages = []  # Reset history

    # Input box for user query
    user_query = st.text_input("Ask a medical question:")

    if st.button("Send") and user_query.strip():
        # Append user query to history
        st.session_state.messages.append({"role": "user", "content": user_query})

        # Generate response
        response = query_pipeline(user_query)
        st.session_state.messages.append({"role": "assistant", "content": response})

    # Display chat history
    st.write("### Chat History")
    for message in st.session_state.messages:
        if message["role"] == "user":
            st.markdown(f"**You:** {message['content']}")
        else:
            st.markdown(f"**Assistant:** {message['content']}")

if __name__ == "__main__":
    main()
"""

# Save code to a Python script
with open("DoctorGPTBot_app.py", "w") as f:
    f.write(streamlit_code)
print("Streamlit code saved as DoctorGPTBot_app.py!")

Streamlit code saved as DoctorGPTBot_app.py!


In [63]:
!pip install pyngrok

In [64]:
!ngrok config add-authtoken 2vXPJtqghyvoldzhiCLevVZd9rF_5zG4YqqH4mqvgaXjj1qXt

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml                                


In [80]:
from pyngrok import ngrok
import subprocess

# Start Streamlit in the background
streamlit_process = subprocess.Popen(["streamlit", "run", "DoctorGPTBot_app.py"])

# Open an Ngrok tunnel for port 8502
public_url = ngrok.connect(8502)
print("Public URL:", public_url)


Public URL: NgrokTunnel: "https://fd37-34-171-175-28.ngrok-free.app" -> "http://localhost:8502"
